Quickstart
======

Here is a small example on how to do Bayesian model selection.

Step by step explanation
----------------------------

### Defining a model


To do model selection, we first need some models. A model, in the simplest case,
is just a callable which takes a single `dict` as input and returns a single `dict` as output. The keys of the input dictionary are the parameters of the model, the output
keys denote the summary statistics.
Here, the `dict` is passed as `parameters` and has the entry `x`, which denotes the mean of a Gaussian.
It returns the observed summary statistics `y`, which is just the sampled value.

In [1]:
import sys 
sys.path.append('../../')
from model_comparison.utils import *
from model_comparison.mdns import *
from model_comparison.models import PoissonModel, NegativeBinomialModel

In [2]:
sample_size = 5
ntest = 10

k2 = 2.
theta2 = 1.0

k3 = 2.
theta3 = 2. 

# then the scale of the Gamma prior for the Poisson is given by
theta1 = 2.0
k1 = (k2 * theta2 * k3 * theta3) / theta1
print(k1)


model_poisson = PoissonModel(sample_size=sample_size, n_workers=2)
model_nb = NegativeBinomialModel(sample_size=sample_size, n_workers=2)

4.0


In [3]:
%matplotlib inline
import os
import tempfile

import scipy.stats as st

from pyabc import (ABCSMC, RV, Distribution,
                   PercentileDistanceFunction)

# Define a gaussian model
sigma = 1.


def Poisson_model(parameters):
    x = model_poisson.gen([parameters.lam])
    return {'y': np.array([x.mean(), x.var()])}

def NB_model(parameters): 
    x = model_nb.gen([[parameters.k, parameters.theta]])
    return {'y': np.array([x.mean(), x.var()])}

For model selection we usually have more than one model.
These are assembled in a list. We
require a Bayesian prior over the models.
The default is to have a uniform prior over the model classes.
This concludes the model definition.

In [4]:
# We define two models, but they are identical so far
models = [Poisson_model, NB_model]


# Define priors 
prior1 = Distribution.from_dictionary_of_dictionaries(dict(lam={'type': 'gamma', 'kwargs': {'a':k1, 'scale': theta1}}))

prior2 = Distribution.from_dictionary_of_dictionaries(dict(k={'type': 'gamma', 'kwargs': {'a':k2, 'scale': theta2}}, 
                                                     theta={'type': 'gamma', 'kwargs': {'a':k3, 'scale': theta3}}))

parameter_priors = [prior1, prior2]

### Configuring the ABCSMC run

Having the models defined, we can plug together the `ABCSMC` class.
We need a distance function,
to measure the distance of obtained samples.

In [5]:
# We plug all the ABC options together
abc = ABCSMC(
    models, parameter_priors,
    PercentileDistanceFunction(measures_to_use=["y"]))

### Setting the observed data

Actually measured data can now be passed to the ABCSMC.
This is set via the `new` method, indicating that we start
a new run as opposed to resuming a stored run (see the "resume stored run" example).
Moreover, we have to set the output database where the ABC-SMC run
is logged.

In [6]:
# y_observed is the important piece here: our actual observation.
y_observed = [ 3.4 ,  5.04]
# and we define where to store the results
db_path = ("sqlite:///" +
           os.path.join(tempfile.gettempdir(), "test.db"))
abc_id = abc.new(db_path, {"y": y_observed})

INFO:History:Start <ABCSMC(id=1, start_time=2018-12-13 10:29:27.729556, end_time=None)>
Process Process-1:
Process Process-2:
Traceback (most recent call last):
Process Process-3:
  File "/home/janfb/anaconda3/envs/sbi/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/janfb/anaconda3/envs/sbi/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/home/janfb/anaconda3/envs/sbi/lib/python3.7/site-packages/pyabc/sampler/multicore_evaluation_parallel.py", line 24, in work
    new_sim = simulate_one()
Traceback (most recent call last):
  File "/home/janfb/anaconda3/envs/sbi/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/janfb/anaconda3/envs/sbi/lib/python3.7/site-packages/pyabc/smc.py", line 437, in simulate_one
    t, theta, self.summary_statistics)
  File "/home/janfb/anaconda3/envs/sbi/lib/python3.7/mu

  File "/home/janfb/anaconda3/envs/sbi/lib/python3.7/site-packages/pyabc/smc.py", line 437, in simulate_one
    t, theta, self.summary_statistics)
Traceback (most recent call last):
  File "/home/janfb/anaconda3/envs/sbi/lib/python3.7/site-packages/pyabc/model.py", line 115, in summary_statistics
    raw_data = self.sample(pars)
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process Process-11:
  File "/home/janfb/anaconda3/envs/sbi/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/janfb/anaconda3/envs/sbi/lib/python3.7/site-packages/pyabc/model.py", line 259, in sample
    return self.sample_function(pars)
  File "/home/janfb/anaconda3/envs/sbi/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/janfb/anaconda3/envs/sbi/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  Fi

  File "/home/janfb/anaconda3/envs/sbi/lib/python3.7/site-packages/pyabc/model.py", line 115, in summary_statistics
    raw_data = self.sample(pars)
  File "/home/janfb/anaconda3/envs/sbi/lib/python3.7/multiprocessing/context.py", line 119, in Pool
    context=self.get_context())
  File "/home/janfb/anaconda3/envs/sbi/lib/python3.7/multiprocessing/pool.py", line 177, in __init__
    self._repopulate_pool()
  File "/home/janfb/anaconda3/envs/sbi/lib/python3.7/site-packages/pyabc/sampler/multicore_evaluation_parallel.py", line 24, in work
    new_sim = simulate_one()
  File "/home/janfb/anaconda3/envs/sbi/lib/python3.7/multiprocessing/context.py", line 119, in Pool
    context=self.get_context())
  File "/home/janfb/anaconda3/envs/sbi/lib/python3.7/multiprocessing/context.py", line 119, in Pool
    context=self.get_context())
  File "/home/janfb/anaconda3/envs/sbi/lib/python3.7/multiprocessing/pool.py", line 177, in __init__
    self._repopulate_pool()
Traceback (most recent call last):


  File "/home/janfb/anaconda3/envs/sbi/lib/python3.7/multiprocessing/process.py", line 110, in start
    'daemonic processes are not allowed to have children'
  File "/home/janfb/anaconda3/envs/sbi/lib/python3.7/site-packages/pyabc/model.py", line 259, in sample
    return self.sample_function(pars)
  File "/home/janfb/anaconda3/envs/sbi/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
Process Process-16:
AssertionError: daemonic processes are not allowed to have children
  File "/home/janfb/anaconda3/envs/sbi/lib/python3.7/multiprocessing/pool.py", line 177, in __init__
    self._repopulate_pool()
  File "<ipython-input-3-da275ed490b4>", line 15, in Poisson_model
    x = model_poisson.gen([parameters.lam])
  File "/home/janfb/anaconda3/envs/sbi/lib/python3.7/site-packages/pyabc/sampler/multicore_evaluation_parallel.py", line 24, in work
    new_sim = simulate_one()
  File "../../model_comparison/models.py", line 32, in gen
    p 

  File "../../model_comparison/models.py", line 32, in gen
    p = Pool(processes=self.n_workers)
  File "/home/janfb/anaconda3/envs/sbi/lib/python3.7/multiprocessing/process.py", line 110, in start
    'daemonic processes are not allowed to have children'
  File "/home/janfb/anaconda3/envs/sbi/lib/python3.7/multiprocessing/context.py", line 119, in Pool
    context=self.get_context())
Traceback (most recent call last):
  File "/home/janfb/anaconda3/envs/sbi/lib/python3.7/multiprocessing/pool.py", line 177, in __init__
    self._repopulate_pool()
AssertionError: daemonic processes are not allowed to have children
  File "/home/janfb/anaconda3/envs/sbi/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/janfb/anaconda3/envs/sbi/lib/python3.7/multiprocessing/pool.py", line 238, in _repopulate_pool
    self._wrap_exception)
  File "/home/janfb/anaconda3/envs/sbi/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._a

  File "/home/janfb/anaconda3/envs/sbi/lib/python3.7/multiprocessing/pool.py", line 238, in _repopulate_pool
    self._wrap_exception)
  File "/home/janfb/anaconda3/envs/sbi/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/janfb/anaconda3/envs/sbi/lib/python3.7/multiprocessing/pool.py", line 257, in _repopulate_pool_static
    w.start()
  File "/home/janfb/anaconda3/envs/sbi/lib/python3.7/site-packages/pyabc/sampler/multicore_evaluation_parallel.py", line 24, in work
    new_sim = simulate_one()
  File "/home/janfb/anaconda3/envs/sbi/lib/python3.7/multiprocessing/process.py", line 110, in start
    'daemonic processes are not allowed to have children'
  File "/home/janfb/anaconda3/envs/sbi/lib/python3.7/site-packages/pyabc/smc.py", line 437, in simulate_one
    t, theta, self.summary_statistics)
Process Process-24:
AssertionError: daemonic processes are not allowed to have children
  File "/home/janfb/anaconda3/envs/

ProcessError: At least one worker is dead.

The `new` method returns an id, which is the id of the
ABC-SMC run in the database.
We're not usint this id for now.
But it might be important when you load the stored data or want
to continue an ABC-SMC run in the case of having more than one
ABC-SMC run stored in a single database.

In [ ]:
print("ABC-SMC run ID:", abc_id)

### Running the ABC

We run the `ABCSMC` specifying the epsilon value at which to terminate.
The default epsilon strategy is the `pyabc.epsilon.MedianEpsilon`.
Whatever is reached first, the epsilon or the maximum number allowed populations,
terminates the ABC run. The method returns a `pyabc.storage.History` object, which
can, for example, be queried for the posterior probabilities.

In [ ]:
# We run the ABC until either criterion is met
history = abc.run(minimum_epsilon=0.0001, max_nr_populations=20)

Note that the history object is also always accessible from the abcsmc object:

In [ ]:
history is abc.history

The `pyabc.storage.History>` object can, for example,
be queried for the posterior probabilities in the populations:

In [ ]:
history.total_nr_simulations

In [ ]:
# Evaluate the model probabililties
model_probabilities = history.get_model_probabilities()
model_probabilities

And now, let's visualize the results:

In [ ]:
model_probabilities.plot.bar();

So model 1 is the more probable one. Which is expected as it was centered at 1 and the observed data was also 1, whereas model 0 was centered at 0, which is farther away from the observed data. 